# Import Libraries

In [1]:
from swarmintelligence import WhaleOptimizationAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/WOA/optimal_params.pkl', 'rb'))
optimal_params

{'n_whales': 25, 'maxIteration': 100, 'constanta': 5}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['mMasi_entropy']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_g = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_b = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
930.7811479817464  |  2436.206106275086  |  1787.8210861061968
930.7683528911109  |  2436.3584799702053  |  1787.8950628016757
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.206106275086  |  1787.9236630530743
930.7811479817464  |  2436.15956953858  |  1787.9236630530743
930.7047355196313  |  2436.3584799702053  |  1787.9236630530743
930.6574285043455  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.8950628016757
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.8210861061968
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7683528911109  |  2436.3584799702053  |  1787.9236630530743
930.7811

1004.7658946943336  |  2393.6040262694814  |  2181.6249270312555
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.7658946943336  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.813810795207
1004.9139322682681  |  2393.647254097627  |  2181.813810795207
1004.7888338820939  |  2393.6040262694814  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.813810795207
1004.9139322682681  |  2393.408017759868  |  2181.813810795207
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2180.6552335218385
1004.9139322682681  |  2393.647254097627  |  2181.813810795207
1004.7326101372672  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.6040262694814  |  2181.8670821363294
1004.7888338820939  |  2393.647254097627  |  2181.813810795207
1004.2574298603836  |  2393.647254097627  |

2947.1042090279807  |  2947.368844954497  |  2947.3859621212478
2947.3859621212478  |  2947.109394071251  |  2946.7894497295792
2946.7758196466157  |  2947.3859621212478  |  2947.2405617700765
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.2405617700765  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.109394071251
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.368844954497  |  2947.2405617700765  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.2405617700765
2947.3859621212478  |  2946.602171907965  |  2947.3859621212478
2947.3859621212478  |  2947.109394071251  |  2947.2405617700765
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.385

3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.791456113383  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.7876103618587
3359.8307253277617  |  2004.6997362709596  |  2640.869259570156
3359.8307253277617  |  2004.4553143805483  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8307253277617  |  2004.6997362709596  |  2640.869259570156
3359.791456113383  |  2004.6997362709596  |  2640.6421806261087
3359.8652306428266  |  2004.6997362709596  |  2640.4291312174682
3359.743500726603  |  2004.6997362709596  |  2640.869259570156
3359.8307253277617  |  2004.6997362709596  |  2640.1531263224942
3359.2611471402915  |  2004.6997362709596  |  2639.4012989941702
3359.8307253277617  |  2004.4553143805

1656.2011866877924  |  5203.798361807061  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.813651622319  |  1759.2175161838823
1656.1370985034855  |  5203.927265675553  |  1758.9276274618476
1656.0552574905123  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.1370985034855  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.1225669032967
1656.1590831079752  |  5203.798361807061  |  1759.116379196032
1656.2011866877924  |  5203.555551247959  |  1759.2175161838823
1656.0997728044963  |  5203.927265675553  |  1436.6018548271252
1656.2011866877924  |  5203.927265675553  |  1759.1225669032967
1656.2011866877924  |  5203.927265675553  |  1436.6018548271252
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.1370985034855  |  5203.927265675553 

5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.418808324037  |  5636.411887253815  |  1589.872409058858
5640.428003063228  |  5636.5693811079145  |  1589.872409058858
5640.428003063228  |  5636.631173704616  |  1589.8493731099352
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.585936244663  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.8493731099352
5640.418808324037  |  5636.631173704616  |  1589.259508894953
5640.428003063228  |  5636.585936244663  |  1589.730962403627
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.386543996076  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.585936244663  |  1589.8493731099352
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.730962403627
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640

## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,mMasi_entropy,max,"[121, 178]","[94, 166]","[125, 182]",930.752119,2436.331401,1787.886726,...,"[[125, 255, 255, 255, 255, 255, 255, 255, 255,...","[930.7811479817464, 930.7683528911109, 930.781...","[2436.206106275086, 2436.3584799702053, 2436.3...","[1787.8210861061968, 1787.8950628016757, 1787....","[4.146122694015503, 3.1245453357696533, 3.7219...","[2302.6959177652993, 2295.0145123799643, 2293....","[47.98641388732127, 47.906309734522075, 47.893...","[14.50843769823287, 14.522949247520984, 14.525...","[0.7812513178831074, 0.7810401305114288, 0.780...","[0.9409462468274888, 0.9409424199537616, 0.941..."
1,AirplaneF16.tiff,3,mMasi_entropy,max,"[105, 156, 195]","[74, 135, 187]","[122, 172, 201]",970.970345,2530.506924,1856.265196,...,"[[122, 255, 201, 255, 255, 201, 201, 201, 201,...","[970.9275559653463, 971.1290558159712, 971.123...","[2534.062306243562, 2533.8703688101477, 2533.9...","[1856.1459129504183, 1856.2530976775427, 1856....","[3.9570398330688477, 3.8516459465026855, 3.323...","[1595.80828221639, 1601.027500152588, 1584.648...","[39.947569165299534, 40.01284169054465, 39.807...","[16.10099646144417, 16.086815691972372, 16.131...","[0.7375193992346424, 0.7367027046987404, 0.736...","[0.958241820402186, 0.9580207398544308, 0.9583..."
2,AirplaneF16.tiff,4,mMasi_entropy,max,"[82, 125, 168, 199]","[54, 109, 159, 199]","[107, 147, 179, 203]",992.437314,2578.223604,1901.251200,...,"[[107, 203, 203, 255, 203, 203, 203, 203, 203,...","[994.8275257587417, 993.6182247399881, 994.723...","[2586.544519886589, 2587.359182922305, 2534.07...","[1900.6682984194115, 1901.7843956176373, 1900....","[3.9945526123046875, 3.886420726776123, 3.9640...","[1164.3282864888508, 1180.1918093363445, 1415....","[34.1222550029867, 34.35391985401876, 37.61811...","[17.470049124189728, 17.411277646069333, 16.62...","[0.7177351442490121, 0.7166992815983848, 0.736...","[0.9693829400078228, 0.9693867496807265, 0.962..."
3,AirplaneF16.tiff,5,mMasi_entropy,max,"[70, 108, 143, 177, 201]","[35, 85, 130, 173, 204]","[104, 139, 166, 189, 207]",1004.667028,2604.601649,1922.044543,...,"[[104, 207, 207, 207, 207, 207, 207, 207, 207,...","[1006.6230307982833, 1005.5974132611116, 1001....","[2614.930794216659, 2586.0545633595857, 2618.9...","[1919.351448257746, 1922.7994601235823, 1922.7...","[3.991699695587158, 4.190209150314331, 3.98849...","[918.4077402750651, 899.0959447224935, 1006.11...","[30.305242785284943, 29.98492862626979, 31.719...","[18.500448255956066, 18.59274322037678, 18.104...","[0.723826858967842, 0.7230053479366805, 0.7265...","[0.9766716461929325, 0.9754909609407777, 0.974..."
4,Lena.png,2,mMasi_entropy,max,"[136, 196]","[78, 146]","[96, 141]",1004.843281,2393.609005,2181.797307,...,"[[141, 141, 141, 141, 141, 141, 141, 141, 141,...","[1004.7658946943336, 1004.9139322682681, 1004....","[2393.6040262694814, 2393.647254097627, 2393.6...","[2181.6249270312555, 2181.8670821363294, 2181....","[4.207355260848999, 3.144906759262085, 4.40201...","[1870.8742192586262, 1858.9784520467122, 1861....","[43.25360354072971, 43.115872391112674, 43.141...","[15.410357704379932, 15.43806005098941, 15.432...","[0.6390917288391306, 0.6394589731037388, 0.639...","[0.8957699047497519, 0.8954769944582632, 0.895..."
5,Lena.png,3,mMasi_entropy,max,"[125, 173, 210]","[56, 106, 159]","[80, 109, 146]",1067.696808,2597.735275,2270.142358,...,"[[146, 146, 146, 146, 146, 146, 146, 146, 146,...","[1070.1183744124746, 1070.0460249103655, 1070....","[2597.917277382854, 2597.917277382854, 2597.96...","[2270.4681775396502, 2270.329140692975, 2270.1...","[3.326190710067749, 3.9207587242126465, 3.2304...","[1172.928429921468, 1188.052037556966, 1186.90...","[34.24804271664978, 34.468130752290094, 34.451...","[17.4380

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/WOA/WOA_masi_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/WOA/WOA_masi_RGB_dict.pkl', 'wb'))